### 示例14-1

In [ ]:
from pyexiv2 import Image
img = Image(r"c:\data\photos\IMG_20190823_095622.jpg")
exif = img.read_exif()
exif

### 14-1（从Exif数据中读取经度、纬度和高程）

In [ ]:
#读取纬度
lat_d,lat_m,lat_s = exif["Exif.GPSInfo.GPSLatitude"].split(" ")
lat_d = float(lat_d.split("/")[0])/float(lat_d.split("/")[1])
lat_m = float(lat_m.split("/")[0])/float(lat_m.split("/")[1])
lat_s = float(lat_s.split("/")[0])/float(lat_s.split("/")[1])
lat = lat_d + lat_m/60.0 + lat_s/3600.0
#读取经度
lng_d,lng_m,lng_s = exif["Exif.GPSInfo.GPSLongitude"].split(" ")
lng_d = float(lng_d.split("/")[0])/float(lng_d.split("/")[1])
lng_m = float(lng_m.split("/")[0])/float(lng_m.split("/")[1])
lng_s = float(lng_s.split("/")[0])/float(lng_s.split("/")[1])
lng = lng_d + lng_m/60.0 + lng_s/3600.0
#读取高程
alt = exif["Exif.GPSInfo.GPSAltitude"]
alt = float(alt.split("/")[0])/float(alt.split("/")[1])
print(f"经度：{lng}")
print(f"纬度：{lat}")
print(f"高程：{alt}")

### 示例14-2

In [ ]:
def reverse_geocoding(lat,lng):
    import math 
    import geopandas as gpd
    from shapely.geometry import Point
    from pyproj import Transformer

    #根据方位角返回方位信息函数
    def direction(angle_degree):
        if angle_degree > -22.5 and angle_degree <= 22.5:
            return "东"
        elif angle_degree > 22.5 and angle_degree <= 67.5:
            return "东北"
        elif angle_degree > 67.5 and angle_degree <= 112.5:
            return "北"
        elif angle_degree > 112.5 and angle_degree <= 157.5:
            return "西北"
        elif angle_degree > 157.5 and angle_degree <= 180:
            return "西"
        elif angle_degree > -180 and angle_degree <= 157.5:
            return "西"
        elif angle_degree > -157.5 and angle_degree <= -112.5:
            return "西南"
        elif angle_degree > -112.5 and angle_degree <= -67.5:
            return "南"
        elif angle_degree > -67.5 and angle_degree <= -22.5:
            return "东南"
        else:
            print("角度范围：-180到180")

    #根据输入坐标构建Point对象
    lnglat_xy = Transformer.from_crs(4326, 3857,always_xy=True)
    x,y = lnglat_xy.transform(lng,lat)
    point = Point((x,y))
    #查找最近POI点
    POIs_wgs = gpd.GeoDataFrame.from_file(r'c:\data\tms\POI_WGS.shp')
    POIs_wgs.crs = 4326
    POIs_proj = POIs_wgs.to_crs(3857)
    dist_list = []
    for POI in POIs_proj["geometry"]:
        dist = POI.distance(point)
        dist_list.append(dist)
    POIs_proj["dist"] = dist_list
    min_distance = POIs_proj["dist"].min()
    exp = POIs_proj["dist"] == min_distance
    nearest_POI = POIs_proj[exp].iloc[0]
    #计算输入点与最近POI的方位角及方位
    POI_x = nearest_POI["geometry"].x
    POI_y = nearest_POI["geometry"].y
    angle = math.atan2(y-POI_y,x-POI_x)
    angle_degree =  math.degrees(angle)
    direction = direction(angle_degree)
    #生成并返回地址信息
    distance = int(min_distance)
    name = nearest_POI["NAME"]
    return f"{name}{direction}{distance}米"

#以下是函数调用语句： 
reverse_geocoding(30.349,119.423)

### 示例14-3

In [ ]:
import folium

def photo_info(dirpath):
    import os
    from pyexiv2 import Image
    files = os.listdir(dirpath)
    data = []
    for file in files:
        fname = f"{dirpath}\\{file}"
        img = Image(fname)
        exif = img.read_exif()
        #读取纬度
        lat_d,lat_m,lat_s = exif["Exif.GPSInfo.GPSLatitude"].split(" ")
        lat_d = float(lat_d.split("/")[0])/float(lat_d.split("/")[1])
        lat_m = float(lat_m.split("/")[0])/float(lat_m.split("/")[1])
        lat_s = float(lat_s.split("/")[0])/float(lat_s.split("/")[1])
        lat = lat_d + lat_m/60.0 + lat_s/3600.0
        #读取经度
        lng_d,lng_m,lng_s = exif["Exif.GPSInfo.GPSLongitude"].split(" ")
        lng_d = float(lng_d.split("/")[0])/float(lng_d.split("/")[1])
        lng_m = float(lng_m.split("/")[0])/float(lng_m.split("/")[1])
        lng_s = float(lng_s.split("/")[0])/float(lng_s.split("/")[1])
        lng = lng_d + lng_m/60.0 + lng_s/3600.0
        #读取照片时间
        time = exif["Exif.Image.DateTime"]
        #读取照片宽度
        width = float(exif["Exif.Image.ImageWidth"])
        #读取照片高度
        height = float(exif["Exif.Image.ImageLength"])
        
        data.append([fname,lng,lat,time,width,height])
    return data

def cal_bound(lng,lat,define_length,width,height):
    from pyproj import Transformer
    lnglat_xy = Transformer.from_crs(4326, 3857,always_xy=True)
    x,y = lnglat_xy.transform(lng,lat)
    if (width/height) >= 1:
        maxx = x + define_length
        maxy = y + define_length*(height/width)
    else:
        maxx = x + define_length*(width/height)
        maxy = y + define_length
    xy_lnglat = Transformer.from_crs(3857,4326,always_xy=True)
    maxlng,maxlat = xy_lnglat.transform(maxx,maxy)
    return (maxlng,maxlat)

def reverse_geocoding(lat,lng):
    import math 
    import geopandas as gpd
    from shapely.geometry import Point
    from pyproj import Transformer

    #根据方位角返回方位信息函数
    def direction(angle_degree):
        if angle_degree > -22.5 and angle_degree <= 22.5:
            return "东"
        elif angle_degree > 22.5 and angle_degree <= 67.5:
            return "东北"
        elif angle_degree > 67.5 and angle_degree <= 112.5:
            return "北"
        elif angle_degree > 112.5 and angle_degree <= 157.5:
            return "西北"
        elif angle_degree > 157.5 and angle_degree <= 180:
            return "西"
        elif angle_degree > -180 and angle_degree <= 157.5:
            return "西"
        elif angle_degree > -157.5 and angle_degree <= -112.5:
            return "西南"
        elif angle_degree > -112.5 and angle_degree <= -67.5:
            return "南"
        elif angle_degree > -67.5 and angle_degree <= -22.5:
            return "东南"
        else:
            print("角度范围：-180到180")

    #根据输入坐标构建Point对象
    lnglat_xy = Transformer.from_crs(4326, 3857,always_xy=True)
    x,y = lnglat_xy.transform(lng,lat)
    point = Point((x,y))
    #查找最近POI点
    POIs_wgs = gpd.GeoDataFrame.from_file(r'c:\data\tms\POI_WGS.shp')
    POIs_wgs.crs = 4326
    POIs_proj = POIs_wgs.to_crs(3857)
    dist_list = []
    for POI in POIs_proj["geometry"]:
        dist = POI.distance(point)
        dist_list.append(dist)
    POIs_proj["dist"] = dist_list
    min_distance = POIs_proj["dist"].min()
    exp = POIs_proj["dist"] == min_distance
    nearest_POI = POIs_proj[exp].iloc[0]
    #计算输入点与最近POI的方位角及方位
    POI_x = nearest_POI["geometry"].x
    POI_y = nearest_POI["geometry"].y
    angle = math.atan2(y-POI_y,x-POI_x)
    angle_degree =  math.degrees(angle)
    direction = direction(angle_degree)
    #生成并返回地址信息
    distance = int(min_distance)
    name = nearest_POI["NAME"]
    return f"{name}{direction}{distance}米"

dirpath = "tms_photos"   #定义照片文件夹路径
data = photo_info(dirpath)
image_layer = folium.FeatureGroup(name="image_layer")
marker_layer = folium.FeatureGroup(name="marker_layer")
for record in data:
    fname = record[0]
    lng = record[1]
    lat = record[2]
    time = record[3]
    width = record[4]
    height = record[5]
    define_length = 500  #定义照片在地图上的外接正方形的边长
    max_lng,max_lat = cal_bound(lng,lat,define_length,width,height)
    folium.raster_layers.ImageOverlay(
        image=fname,
        bounds=[[lat,lng],[max_lat,max_lng]]).add_to(image_layer)
    addr = reverse_geocoding(lat,lng)  #返回地址信息
    popup = folium.Popup(f"{addr},<br>{time}",max_width=200)
    folium.Marker(location=(lat,lng),
                  popup=popup,).add_to(marker_layer)
    
tiles = "https://{s}.tile.openstreetmap.de/tiles/osmde/{z}/{x}/{y}.png"
m = folium.Map(tiles=tiles,
               attr="osmde",
               zoom_start=10)
bounds = marker_layer.get_bounds()
m.fit_bounds(bounds)
image_layer.add_to(m)
marker_layer.add_to(m)
folium.LayerControl().add_to(m)
m